In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm_notebook
np.set_printoptions(suppress=True)
from sklearn import preprocessing
from tqdm import tqdm
import decimal
from itertools import combinations
from random import shuffle
from time import sleep
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
import gc
import datetime
from sklearn.model_selection import train_test_split, KFold, GroupKFold
import os
from sklearn.metrics import roc_auc_score
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
import random
import lightgbm as lgb

In [2]:
#Always seed the randomness of this universe
def seed_everything(seed=51):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [3]:
seed_everything(seed=51)

In [4]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [5]:
train_dtypes = pd.read_csv('../input/train6_dtypes.csv')
test_dtypes = pd.read_csv('../input/test6_dtypes.csv')
train_dtypes.index = train_dtypes['TransactionID']
del train_dtypes['TransactionID']
test_dtypes.index = test_dtypes['TransactionID']
del test_dtypes['TransactionID']
train_dtypes = train_dtypes['int32']
test_dtypes = test_dtypes['int32']
print(type(train_dtypes),type(test_dtypes))
train_dtypes = train_dtypes.to_dict()
test_dtypes = test_dtypes.to_dict()
gc.collect()

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


35

In [6]:
%%time
train = pd.read_csv('../input/train6.csv',dtype= train_dtypes)
print("train finished")

train finished
CPU times: user 1min 9s, sys: 2.18 s, total: 1min 11s
Wall time: 1min 11s


In [7]:
%%time
test = pd.read_csv('../input/test6.csv',dtype= test_dtypes)
print("test finished")

test finished
CPU times: user 58.7 s, sys: 1.26 s, total: 59.9 s
Wall time: 59.9 s


In [8]:
train.head()

TransactionID  isFraud  TransactionDT  TransactionAmt  ProductCD  card1  \
0        2987000        0          86400            68.5          4   2386   
1        2987001        0          86401            29.0          4    315   
2        2987002        0          86469            59.0          4    658   
3        2987003        0          86499            50.0          4   3165   
4        2987004        0          86506            50.0          1    631   

   card2  card3  card4  card5  addr1  addr2  dist1  dist2  P_emaildomain  \
0    297      7      1     18     27      2     19     14              0   
1    173      7      2      1     29      2    497     14             17   
2    228      7      4      9     31      2    498     14             36   
3    277      7      2      2     44      2    497     14             54   
4    242      7      2      1     36      2    497     14             17   

   R_emaildomain   C1   C2   C3   C4   C5   C6   C7   C8   C9  C10  C11  C12  \
0              0  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  2.0  0.0   
1              0  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0   
2              0  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0   
3              0  2.0  5.0  0.0  0.0  0.0  4.0  0.0  0.0  1.0  0.0  1.0  0.0   
4              0  1.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  1.0  1.0  0.0   

    C13  C14     D1     D2    D3    D4   D5   D10    D11    D15  M1  M2  M3  \
0   1.0  1.0   14.0    NaN  13.0   NaN  NaN  13.0   13.0    0.0   1   1   1   
1   1.0  1.0    0.0    NaN   NaN   0.0  NaN   0.0    NaN    0.0   2   2   2   
2   1.0  1.0    0.0    NaN   NaN   0.0  NaN   0.0  315.0  315.0   1   1   1   
3  25.0  1.0  112.0  112.0   0.0  94.0  0.0  84.0    NaN  111.0   2   2   2   
4   1.0  1.0    0.0    NaN   NaN   NaN  NaN   NaN    NaN    NaN   2   2   2   

   M4  M5  M6  M7  M8  M9   V1   V2   V3   V4   V5   V6   V7   V8   V9  V10  \
0   2   0   1   2   2   2  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0   
1   0   1   1   2   2   2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2   0   0   0   0   0   0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0   
3   0   1   0   2   2   2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4   3   2   2   2   2   2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   V11  V12  V13  V14  V15  V16  V17  V18  V19  V20  V21  V22  V23  V24  V25  \
0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0   
1  NaN  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0   
2  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0   
3  NaN  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   V26  V27  V28  V29  V30  V31  V32  V33  V34  V35  V36  V37  V38  V39  V40  \
0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  NaN  NaN  NaN  NaN  NaN  NaN   
1  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0   
2  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0   
3  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   V41  V42  V43  V44  V45  V46  V47  V48  V49  V50  V51  V52  V53  V54  V55  \
0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  1.0   
1  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
2  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0   
3  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   V56  V57  V58  V59  V60  V61  V62  V63  V64  V65  V66  V67  V68  V69  V70  \
0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   
1  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  

In [9]:
#sample_submission
sample_submission = pd.read_csv('../input/sample_submission.csv')
print(train.shape,test.shape,sample_submission.shape)

(590540, 1245) (506691, 1244) (506691, 2)


In [10]:
importances = pd.read_csv('../importances/importances10.csv')
print(importances.shape)
importances = importances[:901]
features = [x for x in list(importances['index']) if x not in ['TransactionID','TransactionDT']]
train = train[['isFraud'] + features ]
test = test[features]

(1244, 7)


In [11]:
print(train.shape,test.shape)

(590540, 900) (506691, 899)


In [12]:
importances.head()

index  Fold_1  Fold_2  Fold_3  Fold_4  Fold_5     avg
0   TransactionID    3195    4913    2080    3005    3111  3260.8
1  TransactionAmt    3087    4449    1924    2961    2886  3061.4
2           addr1    2156    3117    1321    2109    2145  2169.6
3           card1    1990    2789    1232    1802    1957  1954.0
4             D15    1638    2518    1193    1648    1795  1758.4

# Training

In [13]:
y = train['isFraud']
del train['isFraud']

In [16]:
########################### Model params
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.03,
                    'num_leaves': 2**8,
                    'max_depth':12,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.5,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':2000,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': 51,
                    'early_stopping_rounds':100, 
                } 

In [17]:
N = 5
kf = KFold(n_splits=N)

importance = pd.DataFrame(np.zeros((train.shape[1], N)), columns=['Fold_{}'.format(i) for i in range(1, N + 1)], index=train.columns)
scores = []
y_pred = np.zeros(test.shape[0])
oof = np.zeros(train.shape[0])

for fold, (trn_idx, val_idx) in enumerate(kf.split(train, y), 1):
    print('Fold {}'.format(fold))
          
    trn_data = lgb.Dataset(train.iloc[trn_idx, :].values, label=y.iloc[trn_idx].values)
    val_data = lgb.Dataset(train.iloc[val_idx, :].values, label=y.iloc[val_idx].values)   
    
    clf = lgb.train(lgb_params, trn_data ,valid_sets=[trn_data, val_data], verbose_eval=100)

    predictions = clf.predict(train.iloc[val_idx, :].values) 
    importance.iloc[:, fold - 1] = clf.feature_importance()
    oof[val_idx] = predictions

    score = roc_auc_score(y.iloc[val_idx].values, predictions)
    scores.append(score)
    print('Fold {} ROC AUC Score {}\n'.format(fold, score))

    y_pred += clf.predict(test) / N
    
    del trn_data, val_data, predictions
    gc.collect()
    
print('Average ROC AUC Score {} [STD:{}]'.format(np.mean(scores), np.std(scores)))

Fold 1


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.972305	valid_1's auc: 0.900967
[200]	training's auc: 0.989079	valid_1's auc: 0.915291
[300]	training's auc: 0.994456	valid_1's auc: 0.920712
[400]	training's auc: 0.997069	valid_1's auc: 0.922535
[500]	training's auc: 0.998542	valid_1's auc: 0.922924
[600]	training's auc: 0.999229	valid_1's auc: 0.923715
Early stopping, best iteration is:
[548]	training's auc: 0.998911	valid_1's auc: 0.924017
Fold 1 ROC AUC Score 0.9240171794244958

Fold 2


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.972179	valid_1's auc: 0.920158
[200]	training's auc: 0.98993	valid_1's auc: 0.931706
[300]	training's auc: 0.994971	valid_1's auc: 0.935918
[400]	training's auc: 0.997375	valid_1's auc: 0.937405
[500]	training's auc: 0.998735	valid_1's auc: 0.938584
[600]	training's auc: 0.999398	valid_1's auc: 0.939184
[700]	training's auc: 0.999754	valid_1's auc: 0.939165
[800]	training's auc: 0.999903	valid_1's auc: 0.939349
Early stopping, best iteration is:
[766]	training's auc: 0.999855	valid_1's auc: 0.939479
Fold 2 ROC AUC Score 0.9394794770515315

Fold 3


...

Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.973994	valid_1's auc: 0.922078
[200]	training's auc: 0.990695	valid_1's auc: 0.931333
[300]	training's auc: 0.995353	valid_1's auc: 0.932612
[400]	training's auc: 0.997682	valid_1's auc: 0.933147
[500]	training's auc: 0.998859	valid_1's auc: 0.933858
[600]	training's auc: 0.999466	valid_1's auc: 0.933431
Early stopping, best iteration is:
[500]	training's auc: 0.998859	valid_1's auc: 0.933858
Fold 3 ROC AUC Score 0.9338580594183157

Fold 4


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.971352	valid_1's auc: 0.936335
[200]	training's auc: 0.9898	valid_1's auc: 0.94671
[300]	training's auc: 0.9952	valid_1's auc: 0.949388
[400]	training's auc: 0.997556	valid_1's auc: 0.949997
[500]	training's auc: 0.998975	valid_1's auc: 0.950771
Early stopping, best iteration is:
[492]	training's auc: 0.998882	valid_1's auc: 0.950867
Fold 4 ROC AUC Score 0.9508673557832544

Fold 5


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.972277	valid_1's auc: 0.914952
[200]	training's auc: 0.990187	valid_1's auc: 0.925333
[300]	training's auc: 0.995371	valid_1's auc: 0.928167
[400]	training's auc: 0.997578	valid_1's auc: 0.929174
Early stopping, best iteration is:
[390]	training's auc: 0.997428	valid_1's auc: 0.929368
Fold 5 ROC AUC Score 0.9293684298048245

Average ROC AUC Score 0.9355181002964844 [STD:0.009210184727468762]


In [18]:
print(roc_auc_score(y,oof))

0.9343207759300032


In [19]:
importance['avg'] = importance.mean(axis=1)

In [20]:
importance = importance.sort_values(by='avg',ascending=False)

In [21]:
importance = importance.reset_index(drop=False)

In [22]:
importance.head()

index  Fold_1  Fold_2  Fold_3  Fold_4  Fold_5     avg
0  TransactionAmt    3627    5313    3338    3104    2162  3508.8
1       dayofyear    3503    4834    3088    3125    2646  3439.2
2           addr1    2365    3628    2061    2273    1590  2383.4
3           card1    2049    2931    1864    1821    1498  2032.6
4             D15    1986    2586    1727    1719    1491  1901.8

In [23]:
importance.to_csv('../importances/importances11.csv',index=False)

In [24]:
sample_submission.head()

TransactionID  isFraud
0        3663549      0.5
1        3663550      0.5
2        3663551      0.5
3        3663552      0.5
4        3663553      0.5

In [25]:
sample_submission['isFraud'] = y_pred

In [26]:
sample_submission.head()

TransactionID   isFraud
0        3663549  0.000864
1        3663550  0.001170
2        3663551  0.001918
3        3663552  0.001183
4        3663553  0.001057

In [27]:
sample_submission.to_csv('../predictions/pred11.csv',index=False)

In [1]:
!kaggle competitions submit -c ieee-fraud-detection -f ../predictions/pred11.csv -m "pred11"

100%|███████████████████████████████████████| 14.1M/14.1M [00:36<00:00, 408kB/s]
Successfully submitted to IEEE-CIS Fraud Detection